## Step 1
 
 Connect to Google Earth Engine using ee. You will need to have a Google Earth Engine account. 

In [63]:
import ee
# ee.Authenticate()
ee.Initialize()

## Step 2

Read in required libraries. 

In [81]:
import os, re
import pandas as pd
from gee_subset import gee_subset
import geopandas as gpd
from datetime import datetime

import folium
from folium.plugins import *

## Step 3

Read in point data using GeoPandas. Should have the latitude and longitude in WGS84 written as attributes in the table. You should also have a unique ID for each point. 

In [98]:
# pnts = gpd.read_file("D:/gee_examples/points.shp")
pnts = gpd.read_file("../data/tmp/all_locations.shp")
pnts = pnts.iloc[0:9]
pnts.head(10)

,id,x,y,geometry
0,1,6.243377,44.063727,POINT (6.24338 44.06373)
1,2,0.915530,45.635729,POINT (0.91553 45.63573)
2,3,4.337412,45.212939,POINT (4.33741 45.21294)
3,4,5.236351,49.261029,POINT (5.23635 49.26103)
4,5,4.702791,45.421763,POINT (4.70279 45.42176)
5,6,2.326722,43.396599,POINT (2.32672 43.39660)
6,7,6.493826,47.477479,POINT (6.49383 47.47748)
7,8,5.706895,48.475121,POINT (5.70690 48.47512)
8,9,5.733635,45.971452,POINT (5.73363 45.97145)


### Leaflet Map

In [96]:
# Create a folium map centered at the mean coordinates
map_center = [pnts['y'].mean(), pnts['x'].mean()]
my_map = folium.Map(
    location=map_center, 
    zoom_start=6, 
    control_scale=True)

# Add World Imagery TileLayer
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                 name='World Imagery', attr='Esri').add_to(my_map)

# Add World Topo Map TileLayer
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                 name='World Topo', attr='Esri').add_to(my_map)

# Create a MarkerCluster layer for the points
marker_cluster = MarkerCluster().add_to(my_map)

# Add points to the MarkerCluster layer
for _, row in pnts.iterrows():
    folium.Marker(location=[row['y'], row['x']],
                  popup=f"ID: {row['id']}",
                  icon=None  # You can customize the icon if needed
                  ).add_to(marker_cluster)

# Add a MiniMap for an overview
minimap = MiniMap(toggle_display=True).add_to(my_map)

# Add a Draw control for user interaction
draw_control = Draw().add_to(my_map)

# Add a Measure control for distance measurement
measure_control = MeasureControl(primary_length_unit='kilometers').add_to(my_map)

# Add layer control with collapsed legend
folium.LayerControl(collapsed=False).add_to(my_map)

# Display the map
my_map

## Step 4

Count the number of points and create a index list to iterate over. 

In [101]:
cnt = len(pnts)
siteSet = list(range(0, cnt, 1))
print(cnt), print(siteSet)

9
[0, 1, 2, 3, 4, 5, 6, 7, 8]


(None, None)

## Get Data

- Iterate over each sample and extract all data within the provided date range. You can provide a band list to extract only a subset of bands. However, you should include the QA bands for later filtering. 
- Note in our example that the third column (Index 2) is the latitude and the second column (Index 1) is the longitude. The fourth colum (Index 3) is a unique ID.
- Each site is written out to a separate CSV with the ID added to the file name. 

**Earliest date:**  2009-12-01 (for now earliest visit, ignoring that we could use data BEFORE that time to assess temporal lags)

**Latest date:** 2022-01-01

### Landsat Data

In [67]:
for i in siteSet:
    df_landsat = gee_subset.gee_subset(product = "LANDSAT/LC08/C02/T1_L2", 
                                bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "QA_PIXEL"], 
                                start_date = "2013-03-18", 
                                end_date = "2022-01-01", 
                                latitude = pnts.iloc[i, 2], 
                                longitude = pnts.iloc[i ,1], 
                                scale = 700)
    
    sid = str(pnts.iloc[i, 0]) 
    df_landsat["SiteID"] = sid
    df_landsat.to_csv("./gee/landsat/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

### Sentinel 2

In [68]:
for i in siteSet:
    df_s2 = gee_subset.gee_subset(product    =  "COPERNICUS/S2_HARMONIZED", 
                                bands      = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8","B8A", "B9", "B11", "B12", "QA60"],
                                start_date = "2015-06-23", 
                                end_date   = "2022-01-01", 
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 10)
    
    sid = str(pnts.iloc[i, 0]) 
    df_s2["SiteID"] = sid
    df_s2.to_csv("./gee/sentinel2/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

### MODIS

#### Vegetation Indexes

In [69]:
for i in siteSet:
    df_modis_vegetation = gee_subset.gee_subset(
        product    =  "MODIS/061/MOD13Q1", 
        bands      = ["NDVI", "EVI", "DetailedQA"],
        start_date = "2015-06-23", 
        end_date   = "2022-01-01", 
        latitude   = pnts.iloc[i, 2], 
        longitude  = pnts.iloc[i, 1], 
        scale      = 250
        )
    
    sid = str(pnts.iloc[i, 0]) 
    df_modis_vegetation["SiteID"] = sid
    df_modis_vegetation.to_csv("./gee/modis-vegetation/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

#### LST

In [70]:
for i in siteSet:
    df_modis_lst = gee_subset.gee_subset(
        product    =  "MODIS/061/MOD11A1", 
        bands      = ["LST_Day_1km", "QC_Day"],
        start_date = "2015-06-23", 
        end_date   = "2022-01-01", 
        latitude   = pnts.iloc[i, 2], 
        longitude  = pnts.iloc[i, 1], 
        scale      = 1000
        )
    
    sid = str(pnts.iloc[i, 0]) 
    df_modis_lst["SiteID"] = sid
    df_modis_lst.to_csv("./gee/modis-lst/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

### ERA5

#### Daily Aggregates

In [71]:
for i in siteSet:
    df_era_daily = gee_subset.gee_subset(product = "ECMWF/ERA5/DAILY", 
                                bands = ["mean_2m_air_temperature", 
                                         "minimum_2m_air_temperature", 
                                         "maximum_2m_air_temperature", 
                                        #  "dewpoint_2m_temperature", 
                                         "total_precipitation"
                                        #  "surface_pressure", 
                                        #  "mean_sea_level_pressure", 
                                        #  "u_component_of_wind_10m",
                                        #  "v_component_of_wind_10m"
                                         ],
                                start_date = "2009-12-01", 
                                end_date   = "2022-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale = 27830)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_daily["SiteID"] = sid
    df_era_daily.to_csv("./gee/era5-daily-aggregates/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

#### Hourly Data

In [72]:
# Define bands to download:
my_bands = [
    # "dewpoint_temperature_2m"
    "temperature_2m"
    ,"skin_temperature"
    ,"soil_temperature_level_1"
    # ,"soil_temperature_level_2"
    # ,"soil_temperature_level_3"
    # ,"soil_temperature_level_4"
    # ,"lake_bottom_temperature"
    # ,"lake_ice_depth"
    # ,"lake_ice_temperature"
    # ,"lake_mix_layer_depth"
    # ,"lake_mix_layer_temperature"
    # ,"lake_shape_factor"
    # ,"lake_total_layer_temperature"
    # ,"snow_albedo"
    # ,"snow_cover"
    # ,"snow_density"
    # ,"snow_depth"
    # ,"snow_depth_water_equivalent"
    # ,"snowfall"
    # ,"snowmelt"
    # ,"temperature_of_snow_layer"
    # ,"skin_reservoir_content"
    # ,"volumetric_soil_water_layer_1"
    # ,"volumetric_soil_water_layer_2"
    # ,"volumetric_soil_water_layer_3"
    # ,"volumetric_soil_water_layer_4"
    # ,"forecast_albedo"
    # ,"surface_latent_heat_flux"
    # ,"surface_net_solar_radiation"
    # ,"surface_net_thermal_radiation"
    # ,"surface_sensible_heat_flux"
    # ,"surface_solar_radiation_downwards"
    # ,"surface_thermal_radiation_downwards"
    # ,"evaporation_from_bare_soil"
    # ,"evaporation_from_open_water_surfaces_excluding_oceans"
    # ,"evaporation_from_the_top_of_canopy"
    # ,"evaporation_from_vegetation_transpiration"
    # ,"potential_evaporation"
    # ,"runoff"
    # ,"snow_evaporation"
    # ,"sub_surface_runoff"
    # ,"surface_runoff"
    ,"total_evaporation"
    # ,"u_component_of_wind_10m"
    # ,"v_component_of_wind_10m"
    # ,"surface_pressure"
    ,"total_precipitation"
    ,"leaf_area_index_high_vegetation"
    ,"leaf_area_index_low_vegetation"
    # ,"snowfall_hourly"
    # ,"snowmelt_hourly"
    # ,"surface_latent_heat_flux_hourly"
    # ,"surface_net_solar_radiation_hourly"
    # ,"surface_net_thermal_radiation_hourly"
    # ,"surface_sensible_heat_flux_hourly"
    # ,"surface_solar_radiation_downwards_hourly"
    # ,"surface_thermal_radiation_downwards_hourly"
    # ,"evaporation_from_bare_soil_hourly"
    # ,"evaporation_from_open_water_surfaces_excluding_oceans_hourly"
    # ,"evaporation_from_the_top_of_canopy_hourly"
    # ,"evaporation_from_vegetation_transpiration_hourly"
    # ,"potential_evaporation_hourly"
    # ,"runoff_hourly"
    # ,"snow_evaporation_hourly"
    # ,"sub_surface_runoff_hourly"
    # ,"surface_runoff_hourly"
    # ,"total_evaporation_hourly"
    # ,"total_precipitation_hourly
]

Due to quota limitations, only 2 years at once can be downloaded.

In [ ]:
# 2009-2011
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2009-12-01", 
                                end_date   = "2011-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("gee/era5-hourly/2009-2011/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

In [ ]:
# 2011-2013
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2011-12-01", 
                                end_date   = "2013-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("gee/era5-hourly/2011-2013/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

In [ ]:
# 2013-2015
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2013-12-01", 
                                end_date   = "2015-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("gee/era5-hourly/2013-2015/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

In [ ]:
# 2015-2017
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2015-12-01", 
                                end_date   = "2017-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("gee/era5-hourly/2015-2017/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")
    

In [ ]:
# 2017-2019
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2017-12-01", 
                                end_date   = "2019-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("gee/era5-hourly/2017-2019/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")


In [ ]:
# 2019-2021
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2019-12-01", 
                                end_date   = "2021-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("gee/era5-hourly/2019-2021/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")


In [ ]:
# 2021-2022
for i in siteSet:
    df_era_hourly = gee_subset.gee_subset(product = "ECMWF/ERA5_LAND/HOURLY", 
                                bands      = my_bands,
                                start_date = "2021-12-01", 
                                end_date   = "2022-01-01",  # Last needed date "2022-01-01"
                                latitude   = pnts.iloc[i, 2], 
                                longitude  = pnts.iloc[i, 1], 
                                scale      = 11132)
    
    sid = str(pnts.iloc[i, 0]) 
    df_era_hourly["SiteID"] = sid
    df_era_hourly.to_csv("./gee/era5-hourly/2021-2022/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

### Land Use

In [ ]:
# No good dataset found so far.

### Precipitation NOAA

In [100]:
for i in siteSet:
    df_noaa = gee_subset.gee_subset(
        product = "NOAA/PERSIANN-CDR", 
        bands      = ["precipitation"],
        start_date = "2009-01-01", 
        end_date   = "2022-01-01", 
        latitude   = pnts.iloc[i, 2], 
        longitude  = pnts.iloc[i, 1], 
        scale      = 27830
        )
    
    sid = str(pnts.iloc[i, 0]) 
    df_noaa["SiteID"] = sid
    df_noaa.to_csv("./gee/noaa/" + "site_" + str(pnts.iloc[i, 0]) + ".csv")

## Data Processing

In [104]:
def load_and_combine_csv_files(folder_path):
    """
    Load multiple CSV files from a folder and combine them into one DataFrame.

    Parameters:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - combined_df (pd.DataFrame): Combined DataFrame.
    """
    # Get a list of all CSV files in the folder
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    # Initialize an empty list to store individual DataFrames
    dfs = []

    # Iterate through each CSV file and read its content into a DataFrame
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(file_path)
        dfs.append(df)

    # Combine all DataFrames into one
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

### Landsat

In [111]:
folder_path = "./gee/landsat"
data_landsat = load_and_combine_csv_files(folder_path)
data_landsat.head()


,Unnamed: 0,id,longitude,latitude,date,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,QA_PIXEL,product,SiteID
0,0,LC08_199028_20130414,0.914934,45.636662,2013-04-14 10:43:26.562000128,8232.0,8391.0,9463.0,9159.0,20512.0,14592.0,11189.0,21824.0,LANDSAT/LC08/C02/T1_L2,2
1,1,LC08_199028_20130516,0.914934,45.636662,2013-05-16 10:43:36.624000000,7308.0,7407.0,7842.0,7532.0,11149.0,8357.0,7827.0,23888.0,LANDSAT/LC08/C02/T1_L2,2
2,2,LC08_199028_20130601,0.914934,45.636662,2013-06-01 10:43:40.846999808,30021.0,30066.0,29378.0,29377.0,32659.0,25138.0,21899.0,22280.0,LANDSAT/LC08/C02/T1_L2,2
3,3,LC08_199028_20130719,0.914934,45.636662,2013-07-19 10:43:35.515000064,37687.0,37886.0,37055.0,37142.0,38875.0,27618.0,22434.0,55052.0,LANDSAT/LC08/C02/T1_L2,2
4,4,LC08_199028_20130804,0.914934,45.636662,2013-08-04 10:43:38.230000128,8313.0,8481.0,9474.0,9196.0,20789.0,15527.0,11397.0,21824.0,LANDSAT/LC08/C02/T1_L2,2


In [112]:
data_landsat.tail()

,Unnamed: 0,id,longitude,latitude,date,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,QA_PIXEL,product,SiteID
1488,510,LC08_197028_20210913,4.700435,45.422863,2021-09-13 10:29:32.792999936,8265.0,8582.0,9605.0,9522.0,17921.0,14421.0,11293.0,21824.0,LANDSAT/LC08/C02/T1_L2,5
1489,511,LC08_197028_20210929,4.700435,45.422863,2021-09-29 10:29:36.845999872,8583.0,8714.0,9694.0,9389.0,20552.0,15527.0,11569.0,21824.0,LANDSAT/LC08/C02/T1_L2,5
1490,512,LC08_197028_20211015,4.700435,45.422863,2021-10-15 10:29:41.438999808,8008.0,8222.0,9135.0,8886.0,17070.0,12898.0,10268.0,21824.0,LANDSAT/LC08/C02/T1_L2,5
1491,513,LC08_197028_20211031,4.700435,45.422863,2021-10-31 10:29:41.724000000,17870.0,18049.0,19278.0,19560.0,24441.0,21060.0,19190.0,22280.0,LANDSAT/LC08/C02/T1_L2,5
1492,514,LC08_197028_20211218,4.700435,45.422863,2021-12-18 10:29:36.005000192,42637.0,42743.0,40455.0,40600.0,39274.0,25864.0,21235.0,22280.0,LANDSAT/LC08/C02/T1_L2,5


In [121]:
print(sorted(data_landsat['QA_PIXEL'].unique()))

[21762.0, 21824.0, 21890.0, 21952.0, 22018.0, 22080.0, 22280.0, 23826.0, 23888.0, 24082.0, 24144.0, 29986.0, 30048.0, 30242.0, 54596.0, 55052.0, nan, 54534.0, 54790.0, 56660.0, 56854.0, 56916.0]


In [114]:
# Remove pixels with bad data quality
21824 << 2

87296

In [115]:
high_quality_mask = 0b0111111110000000  # Binary representation based on the criteria

# Check if the bitmask result meets the criteria for high-quality data
is_high_quality = (bitmask_result & high_quality_mask) == high_quality_mask

NameError: name 'bitmask_result' is not defined